In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import re
from random import randint
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle
import requests
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## Initiate webdriver

In [ ]:
browser = webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver')
browser.get('https://www.betterwood.de/shop/')

In [ ]:
# getting keywords
keywords = pd.read_csv('Keywords.csv', index_col=[0])

### Getting page links

In [ ]:
path = '//*[@class="product-categories"]//a[@href]'
elems = browser.find_elements_by_xpath(path)
categories=[]
for elem in elems:
    if '/holz/' in elem.get_attribute("href"):
        categories.append(elem.get_attribute("href"))

In [ ]:
page_links=[]
for cat in categories:
    browser.get(cat)
    sleep(2)
    elems = browser.find_elements_by_xpath("//a[@href]")
    for elem in elems:
        if 'kaufen' in elem.get_attribute("href"):
            page_links.append(elem.get_attribute("href"))
len(page_links)

In [ ]:
page_links_clean = []
for x in page_links:
    if 'aluminium' not in x and x not in page_links_clean:
        page_links_clean.append(x)
len(page_links_clean)

In [ ]:
with open('BetterWoodLinks.pkl', 'wb') as f:
    pickle.dump(page_links_clean, f)

In [ ]:
with open('BetterWoodLinks.pkl', 'rb') as f:
    page_links_clean = pickle.load(f)

### Functions inside the crawler

In [ ]:
def main_data():
    # main price
    path = '//*[@class="woocommerce-Price-amount amount"]'
    browser.implicitly_wait(3)
    main_price = browser.find_element_by_xpath(path).text

    if not re.findall(r'\d+', main_price):
        path = '/html/body/div[1]/main/div/div[2]/div[1]/div[1]/div/div[2]/div[1]/p/span'
        main_price = browser.find_element_by_xpath(path).text

        if not re.findall(r'\d+', main_price):
            path = '/html/body/div[1]/main/div/div[2]/div/div[1]/div/div[2]/div[1]/p/span/span'
            main_price = browser.find_element_by_xpath(path).text

    # main header
    path = '//*[@class="product-title product_title entry-title"]'
    main_header = browser.find_element_by_xpath(path).text

    # delivery
    try:
        sleep(1)
        delivery = np.nan
        path = '//*[@class="cuzzy-edd-description"]'
        delivery = browser.find_element_by_xpath(path).text
    except:
        pass
    try:
        # additional info
        path = '//*[@id="tab-title-additional_information"]/a'
        browser.find_element_by_xpath(path).click()

    except:
        pass

    # main table
    sleep(1)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')


    table = soup.findAll("div", { "class" : "product-short-description" })
    main_table_data = {}
    for el in table:
        rows = el.find_all('tr')
        for row in rows:
            if not re.findall(r'(ab \d+\.\d+ €)', row.find_all('td')[0].text):
                key = row.find_all('td')[0].text
                value = row.find_all('td')[1].text
                main_table_data[key] = value

    main_table = pd.DataFrame(main_table_data, index=[0])
    main_table['Main Price'] = main_price
    main_table['Main Header'] = main_header
    main_table['Main Delivery'] = delivery
    main_table['Index'] = i
    main_table['URL'] = url
    main_table['Matched Keywords'] = [{}]
    main_table['Other Keywords'] = [{}]

    # additional data
    table = soup.findAll("table", { "class" : "woocommerce-product-attributes shop_attributes" })
    second_table_data = {}
    for el in table:
        rows = el.find_all('tr')
        for row in rows:
            if not re.findall(r'(ab \d+\.\d+ €)', row.find_all('td')[0].text):
                key = row.find_all('th')[0].text
                value = row.find_all('td')[0].text
                second_table_data[key] = value
    second_table = pd.DataFrame(second_table_data, index=[0])
    rename_cols = {}
    for col in second_table.columns:
        new_col = 'Second_'+col
        rename_cols[col] = new_col
    second_table = second_table.rename(columns=rename_cols)
    main_table = pd.concat([main_table, second_table], axis=1)
    
    # preprocessing to match and extract keywords
    try:
        main_table['Oberfläche'] = main_table['Oberfläche'].fillna(main_table['Second_Oberfläche'])
        main_table['Qualität'] = main_table['Qualität'].fillna(main_table['Second_Qualität'])
        main_table['Oberfläche'] = main_table['Oberfläche']  + ', ' + main_table['Behandlung']
    except:
        pass
    
    rename_cols = {'Second_Produktart':'Product Type', 
                   'Second_Holzart':'Sec_Wood_Type',
                   'Second_Herkunft':'Origin', 
                   'Second_Farbe':'Color',
                   'Option_Grosse':'Option_Size',
                   'Second_Größe':'Other Dimensions',
                   'Holzart':'Wood type',
                   'Option':'Option_Size',
                   'Oberfläche':'Surfase',
                   'Qualität':'Quality',
                   'Breite':'Possible Width',
                   'Länge':'Possible Length',
                   'Main Header':'Product Title',
                   'Stärke':'Thickness',
                   'Trocknung':'Drying Method'}
    main_table = main_table.rename(columns=rename_cols)
    
    # Geting Min/Max Length/Width for custom input
    try:
        length_slpit = main_table['Länge'].str.replace(r'(\D+)',' ').str.replace(r'^ | $','').str.replace(r' +',' ').str.split(r' ', expand=True)
        width_slpit = main_table['Breite'].str.replace(r'(\D+)',' ').str.replace(r'^ | $','').str.replace(r' +',' ').str.split(r' ', expand=True)

        if length_slpit.shape[1] == 2:
            main_table['Länge Min'] = length_slpit[0].astype(int)
            main_table['Länge Max'] = length_slpit[1].astype(int)
        elif length_slpit.shape[1] == 1:
            main_table['Länge Min'] = 5
            main_table['Länge Max'] = length_slpit[0].astype(int)

        if width_slpit.shape[1] == 2:
            main_table['Breite Min'] = width_slpit[0].astype(int)
            main_table['Breite Max'] = width_slpit[1].astype(int)
        elif width_slpit.shape[1] == 1:
            main_table['Breite Min'] = 5
            main_table['Breite Max'] = width_slpit[0].astype(int)

        len_min = main_table['Länge Min'].loc[0]
        len_max = main_table['Länge Max'].loc[0]
        wid_min = main_table['Breite Min'].loc[0]
        wid_max = main_table['Breite Max'].loc[0]

    except:
        len_min = np.nan
        len_max = np.nan
        wid_min = np.nan
        wid_max = np.nan
        pass
    
    return main_table, len_min, len_max, wid_min, wid_max

### Crawling options

#### Option 1. Select length

In [ ]:
def select_length():
    
    options_data = {}
    options_data['Option_Length'] = []
    options_data['Option_Price'] = []
    options_data['SKU'] = []
    
    for n in range(1, len(length_tags)+1):
        sleep(0.5)
        option_path = '//*[@id="pa_laenge"]/option[{}]'.format(n)
        browser.find_element_by_xpath(option_path).click()
        length = browser.find_element_by_xpath(option_path).text
        if not re.findall(r'\d', length):
                continue
        # price
        price_path = '//*[@class="woocommerce-variation-description"]/h2'
        price = browser.find_element_by_xpath(price_path).text

        # sku
        sku_path = '//*[@class="sku"]'
        try:
            sleep(0.1)
            sku_num = browser.find_element_by_xpath(sku_path).text
        except:
            sku_num = np.nan
        options_data['SKU'].append(sku_num)
        options_data['Option_Length'].append(length)
        options_data['Option_Price'].append(price)

    option_table = pd.DataFrame(options_data)
    print('Option Table Size', option_table.shape)
    return option_table

#### Option 2. Select length and width

In [ ]:
def select_lenth_width():
    options_data = {}
    options_data['Option_Width'] = []
    options_data['Option_Length'] = []
    options_data['Option_Price'] = []
    options_data['SKU'] = []

    sleep(1)
    for w in range(1, len(width_tags)+1):

        # width
        width = np.nan
        option_width = '//*[@id="pa_breite"]/option[{}]'.format(w)
        browser.find_element_by_xpath(option_width).click()
        width = browser.find_element_by_xpath(option_width).text
        if not re.findall(r'\d', width):
            continue

        # length
        for l in range(1, len(length_tags)+1):
            length = np.nan
            option_length = '//*[@id="pa_laenge"]/option[{}]'.format(l)
            browser.find_element_by_xpath(option_length).click()
            length = browser.find_element_by_xpath(option_length).text
            
            if not re.findall(r'\d', length):
                    continue
            # price
            sleep(1)
            price = np.nan
            price_path = '//*[@class="woocommerce-variation-description"]/h2'
            price = browser.find_element_by_xpath(price_path).text
            # sku
            try:
                sleep(0.1)
                sku_num = browser.find_element_by_xpath(sku_path).text
            except:
                sku_num = np.nan
            options_data['SKU'].append(sku_num)
            options_data['Option_Width'].append(width)
            options_data['Option_Length'].append(length)
            options_data['Option_Price'].append(price)

    option_table = pd.DataFrame(options_data)
    print('Option Table Size', option_table.shape)
    return option_table

#### Option 3. Select size

In [ ]:
def select_size():
    options_data = {}
    options_data['SKU'] = []
    options_data['Option_Grosse'] = []
    options_data['Option_Price'] = []

    for n in range(1, len(grosse_tags)+1):
        sleep(1)
        option_path = '//*[@id="pa_groesse"]/option[{}]'.format(n)
        browser.find_element_by_xpath(option_path).click()
        option_value = browser.find_element_by_xpath(option_path).text
        if not re.findall(r'\d', option_value):
                continue
        # price
        price_path = '//*[@class="woocommerce-variation-description"]/h2'
        price = browser.find_element_by_xpath(price_path).text
        options_data['Option_Grosse'].append(option_value)
        options_data['Option_Price'].append(price)
        # sku
        sku_path = '//*[@class="sku"]'
        try:
            sleep(0.3)
            sku_num = browser.find_element_by_xpath(sku_path).text
            options_data['SKU'].append(sku_num)
        except:
            options_data['SKU'].append(np.nan)

    option_table = pd.DataFrame(options_data)
    print('Option Table Size', option_table.shape)
    
    return option_table

#### Option 4. Custom Input

In [ ]:
def custom_input():

    options_data = {}
    options_data['Option_Width'] = []
    options_data['Option_Length'] = []
    options_data['Option_Price'] = []
    options_data['SKU'] = []
    
    for width in [x for x in range(wid_min, wid_max+5, 5)]: 

        # width
        browser.find_element_by_xpath(custom_input_width).send_keys(width)


        # length
        for length in [x for x in range(len_min, len_max+5, 5)]: 
            browser.find_element_by_xpath(custom_input_length).send_keys(length)

            # price
            price = np.nan
            sleep(0.5)
            price_path = '//*[@id="price_calculator"]/tbody/tr[4]/td[2]/span/span'
            price = browser.find_element_by_xpath(price_path).text

            # sku
            sku_path = '//*[@class="sku"]'
            try:
                sleep(0.1)
                sku_num = browser.find_element_by_xpath(sku_path).text
            except:
                sku_num = np.nan

            sleep(1)
            browser.find_element_by_xpath(custom_input_length).clear()

            options_data['Option_Width'].append(width)
            options_data['Option_Length'].append(length)
            options_data['Option_Price'].append(price)
            options_data['SKU'].append(sku_num)

        browser.find_element_by_xpath(custom_input_width).clear()

    option_table = pd.DataFrame(options_data)
    print('Option Table Size', option_table.shape)
    return option_table

In [ ]:
def get_keywords(row):
    row['Matched Keywords'].loc[0] = [{}]
    row['Other Keywords'].loc[0] = [{}]
    row = row.fillna(-999)
    matched_cols=[]
    
    for col in ['Product Type', 'Wood type', 'Quality', 'Drying Method','Surfase']:
        for key_word in keywords[col].unique():
            try:
                if fuzz.token_set_ratio(row[col].loc[0], key_word) > 90:
                    row['Matched Keywords'].loc[0][0][row[col].loc[0]] = key_word
                    matched_cols.append(col)
            except:
                pass
    
    if len(matched_cols)==0:
        row['Other Keywords'] = -999
        row['Matched Keywords'] = -999
    
    for col in ['Product Type', 'Wood type', 'Quality', 'Drying Method','Surfase']:
        try:
            if col not in matched_cols and len(matched_cols)>0 and row[col].loc[0]!=-999:
                row['Other Keywords'].loc[0][0][col] = row[col].loc[0]
        except:
                pass
    
    return row

### Crawling on pages and collecting data

In [ ]:
all_products_data = pd.DataFrame([])

In [ ]:
for i in range(2,4):
    url = page_links_clean[i] 
    browser.get(url)

    print('#',i)
    
    main_table, len_min, len_max, wid_min, wid_max = main_data()[0], main_data()[1], main_data()[2], main_data()[3], main_data()[4]
    main_table = get_keywords(main_table)
    if main_table['Matched Keywords'].loc[0] == -999:
        print('No matched keywords', url)
        continue
    else:
        pass
        
    elems = browser.find_elements_by_xpath('//*[@id="pa_laenge"]/option')
    length_tags = [x.get_attribute("value") for x in elems]

    elems = browser.find_elements_by_xpath('//*[@id="pa_breite"]/option')
    width_tags = [x.get_attribute("value") for x in elems]

    elems = browser.find_elements_by_xpath('//*[@id="pa_groesse"]/option')
    grosse_tags = [x.get_attribute("value") for x in elems]

    custom_input_length = '//*[@id="length_needed"]'
    custom_input_width = '//*[@id="width_needed"]'

    try:
        browser.find_element_by_xpath(custom_input_length)
        browser.find_element_by_xpath(custom_input_width)
        print('Inputting custom dimentions')
        option_table = custom_input()
        main_table = main_table.append([main_table]*(option_table.shape[0]-1),ignore_index=True)
        product_data = pd.concat([main_table, option_table], axis=1)
        all_products_data = pd.concat([all_products_data, product_data], axis=0)
        continue
    except:
        pass

    if length_tags and not width_tags:
        print('Choosing length')
        option_table = select_length()
        main_table = main_table.append([main_table]*(option_table.shape[0]-1),ignore_index=True)
        product_data = pd.concat([main_table, option_table], axis=1)
        all_products_data = pd.concat([all_products_data, product_data], axis=0)
        continue

    elif length_tags and width_tags:
        print('Choosing length and width')
        option_table = select_lenth_width()
        main_table = main_table.append([main_table]*(option_table.shape[0]-1),ignore_index=True)
        product_data = pd.concat([main_table, option_table], axis=1)
        all_products_data = pd.concat([all_products_data, product_data], axis=0)
        continue

    elif grosse_tags:
        print('Choosing size')
        option_table = select_size()

        main_table = main_table.append([main_table]*(option_table.shape[0]-1),ignore_index=True)
        product_data = pd.concat([main_table, option_table], axis=1)
        all_products_data = pd.concat([all_products_data, product_data], axis=0)
        continue

    else:
        print('No options to choose')
        sku_path = '//*[@class="sku"]'
        sku_num = browser.find_element_by_xpath(sku_path).text
        main_table['SKU'] = sku_num
        all_products_data = pd.concat([all_products_data, main_table], axis=0, ignore_index=True)